In [1]:
import torch.nn.functional as F
import torch
import pandas as pd
import os

In [2]:
num_classes = 9
input_chunk_length = 120

In [3]:
def dscovr_df(filename):
    df = pd.read_csv(os.path.join(os.path.curdir, "data", filename))
    df = df.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
    df.index = df["0"]
    return df.drop(columns="0")
# print(dscovr_df("data_2016.csv")["k_index_target"].unique().size)

In [4]:
from dscovr_dataset import create_dscovr_dataset
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd

train_df = dscovr_df("data_2017.csv")
X_train, y_train = create_dscovr_dataset(train_df.to_numpy(), lookback=input_chunk_length, num_classes=num_classes)
data_train = data.TensorDataset(X_train, y_train)

test_df = dscovr_df("data_2020.csv")
X_test, y_test = create_dscovr_dataset(test_df.to_numpy(), lookback=input_chunk_length, num_classes=num_classes)
data_test = data.TensorDataset(X_test, y_test)

batch_size = 32
train_loader = data.DataLoader(data_train, shuffle=True, batch_size=batch_size)
test_loader = data.DataLoader(data_test, shuffle=True, batch_size=batch_size * 8)

100%|██████████| 4410/4410 [00:00<00:00, 68978.32it/s]


In [5]:
from transformer_classifier import TransformerClassifier
model = TransformerClassifier(
    num_classes=num_classes,
    input_chunk_length = input_chunk_length,
    d_model = 64,
    input_dim=54,
    nhead = 8,
    dim_feedforward = 256,
    num_layers = 4,
    dropout = 0.5,
    activation = "relu",
    classifier_dropout = 0.5,
).to(device="cuda")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5)

In [6]:
from tqdm import tqdm
from torchmetrics.functional import precision, recall, accuracy

n_epochs = 50
for epoch in range(n_epochs):
    model.train()
    prec, rec, acc = [], [], []
    run_loss = 0
    for X_batch, y_batch in tqdm(train_loader):
        X_batch, y_batch = X_batch.to(device="cuda"), y_batch.to(device="cuda", dtype=torch.float)
        y_pred = model(X_batch)

        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch)

        y_pred = y_pred.argmax(dim=1)
        y_batch = y_batch.argmax(dim=1)
        curr_prec = precision(y_pred, y_batch, "multiclass", num_classes=num_classes, average="macro")
        curr_rec = recall(y_pred, y_batch, "multiclass", num_classes=num_classes, average="macro")
        curr_acc = accuracy(y_pred, y_batch, "multiclass", num_classes=num_classes, average="macro")
        prec.append(curr_prec.item()), rec.append(curr_rec.item()), acc.append(curr_acc.item())
        run_loss += loss.item()

        loss.backward()
        optimizer.step()
    print(f"For index {epoch}: precision: {sum(prec)/len(prec)} recall: {sum(rec)/len(rec)} acc: {sum(acc)/len(acc)}")
    print("--------------------------------")

    model.eval()
    if epoch % 5 == 4:
        prec, rec, acc = [], [], []
        with torch.no_grad():
            for i, (inputs, targets) in tqdm(enumerate(test_loader)):
                inputs, targets = inputs.to(device="cuda"), targets.to(device="cuda")
                y_pred = model(inputs)

                y_pred = y_pred.argmax(dim=1)
                targets = targets.argmax(dim=1)
                curr_prec = precision(y_pred, targets, "multiclass", num_classes=num_classes, average="macro")
                curr_rec = recall(y_pred, targets, "multiclass", num_classes=num_classes, average="macro")
                curr_acc = accuracy(y_pred, targets, "multiclass", num_classes=num_classes, average="macro")
                prec.append(curr_prec.item()), rec.append(curr_rec.item()), acc.append(curr_acc.item())

        # Print accuracy
        print(f"For test: precision: {sum(prec)/len(prec)} recall: {sum(rec)/len(rec)} acc: {sum(acc)/len(acc)}")
        print("--------------------------------")

100%|██████████| 302/302 [00:23<00:00, 13.10it/s]


For index 0: precision: 0.22721151977974846 recall: 0.15172735342620225 acc: 0.15172735342620225
--------------------------------


100%|██████████| 302/302 [00:22<00:00, 13.25it/s]


For index 1: precision: 0.2794771987747475 recall: 0.18005449706010077 acc: 0.18005449706010077
--------------------------------


100%|██████████| 302/302 [00:22<00:00, 13.28it/s]


...

For index 48: precision: 0.5634219154024755 recall: 0.2871630335140307 acc: 0.2871630335140307
--------------------------------


100%|██████████| 302/302 [00:23<00:00, 13.12it/s]


For index 49: precision: 0.5817879694976554 recall: 0.3032151830709533 acc: 0.3032151830709533
--------------------------------


8it [00:01,  4.78it/s]

For test: precision: 0.10868454165756702 recall: 0.1326053524389863 acc: 0.1326053524389863
--------------------------------


In [7]:
from torchmetrics.functional import precision
preds  = torch.tensor([[1, 0], [1, 0], [1, 0], [1, 0]])
target = torch.tensor([[1, 0], [1, 0], [1, 0], [1, 0]])
print(precision(preds, target, task="multiclass", average='micro', num_classes=2))
# precision(preds, target, average='micro')

tensor(1.)
